### Processamento inicial da base de dados MIMIC-IV

O objetivo desse notebook é realizar um processamento das tabelas originais do MIMIC-IV com objetivo de gerar tabelas intermediarias contendo apenas os dados utilizados na análise exploratória

Inicialmente filtramos a tabela `chartevents.csv` que possui diversas informações sobre sobre eventos do hospital inclusive sobre o exame de CK (CPK), identificado pela coluna `itemid` igual a `225634` (pode ser verificado utilizando a tabela `d_items.csv`). Assim, filtramos apenas esses exames. O resultado é salvo na tabela `ck_events.csv`

Como a tabela `chartevents.csv` é muito grande (30gb) ela foi processada em pedaços (chunks).

In [1]:
path_chartevents = "../data/external/chartevents.csv"
outfilename = "../data/interim/ck_events.csv"

In [2]:
import pandas as pd

ck_cpk_id = 225634
chunksize = 10 ** 7  #quantidade de linhas a serem lidas de uma vez

df_final = None
with pd.read_csv(path_chartevents, chunksize=chunksize) as reader:
    for chunk in reader:
        chunk = chunk.loc[chunk['itemid'] == ck_cpk_id]
        if len(chunk) > 0:
            if df_final is None:
                df_final = chunk
            else:
                df_final = pd.concat([df_final,chunk])
                
print("Saving processed table in:", outfilename)
df_final.to_csv(outfilename)

Saving processed table in: ../data/interim/ck_events.csv


In [3]:
#visualizando a tabela gerada
df = pd.read_csv(outfilename)
print(df.head())
print("LEN:", len(df))

   Unnamed: 0  Unnamed: 0.1  subject_id   hadm_id   stay_id  caregiver_id  \
0           0           932    10000980  26913865  39765666           NaN   
1           1         18918    10001884  26184834  37510196           NaN   
2           2         19039    10001884  26184834  37510196           NaN   
3           3         22053    10002155  20345487  32358465           NaN   
4           4         27263    10002155  23822395  33685454           NaN   

             charttime            storetime  itemid  value  valuenum valueuom  \
0  2189-06-27 12:58:00  2189-06-27 14:18:00  225634    263     263.0     IU/L   
1  2131-01-14 03:08:00  2131-01-14 09:18:00  225634    786     786.0     IU/L   
2  2131-01-16 04:02:00  2131-01-16 05:06:00  225634    361     361.0     IU/L   
3  2131-03-10 02:04:00  2131-03-10 03:17:00  225634     99      99.0     IU/L   
4  2129-08-04 17:56:00  2129-08-04 18:51:00  225634    589     589.0     IU/L   

   warning  
0      1.0  
1      1.0  
2      1.0 

#### Informaçoes dos pacientes que fizeram exames de CK (CPK)

A tabela gerada (`ck_events.csv`) possui o identificador do paciente que realizou o exame na coluna `subject_id`. Utilizando essa informação podemos pegar as informações desses pacientes. As informações dos pacientes do MIMIC-IV estão disponiveis na tabela `omr.csv`, que possui informações sobre peso, altura, IMC e pressão arterial. 
O codigo abaixo obtem as informações de todos os pacientes que fizeram algum exame CK (CPK), e salva na tabela `pacients_with_ck.csv`

In [4]:
pacient_info_path = "../data/external/omr.csv"
ck_events_path = "../data/interim/ck_events.csv"
output_pacients_with_ck = "../data/interim/pacients_with_ck.csv"

In [5]:
df_ck = pd.read_csv(ck_events_path)
all_pacients = df_ck['subject_id']

df_pacients = pd.read_csv(pacient_info_path)
df_pacients_with_ck = df_pacients[df_pacients['subject_id'].isin(all_pacients)]

print("Saving processed table in:", output_pacients_with_ck)
df_pacients_with_ck.to_csv(output_pacients_with_ck)

Saving processed table in: ../data/interim/pacients_with_ck.csv


In [6]:
#visualizando a tabela gerada
df = pd.read_csv(output_pacients_with_ck)
print(df.head())
print("LEN:", len(df))

   Unnamed: 0  Unnamed: 0.1  subject_id   chartdate  seq_num     result_name  \
0           0           328    10000980  2185-06-17        1  Blood Pressure   
1           1           329    10000980  2185-06-17        1     BMI (kg/m2)   
2           2           330    10000980  2185-06-17        1    Weight (Lbs)   
3           3           331    10000980  2185-09-17        1     BMI (kg/m2)   
4           4           332    10000980  2185-09-17        1    Weight (Lbs)   

  result_value  
0       150/90  
1         33.6  
2          172  
3         34.0  
4          174  
LEN: 653586


#### Informações sobre aplicações de insulina
Além das informações obtidas no processamento acima, também estamos interessados em verificar se esses pacientes receberam doses de insulina.
A tabela `inputevents.csv` possui informaçoes sobre todos os eventos de entradas de fluido (incluindo insulina). Observando novamente a tabela `d_items.csv`, a insulina aparece com `itemid` com os numeros: 223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619. Esses numeros se referem as insulinas descritas como:  Insulin - 70/30, Insulin - Regular, Insulin - NPH, Insulin - Glargine, Insulin - Humalog 75/25, Insulin - Humalog,Insulin - Novolog, Insulin - U500

A parti dessa tabela é gerada uma nova tabela com o nome de `insulin_inputs_in_ck_pacients.csv` com todos os eventos de aplicação de insulina em pacientes que fizeram algum exame de CK (CPK).

Novamente essa tabela (`inputevents.csv`) é muito grande (2,3GB) assim também foi processada em pedaços (chunks).

In [7]:
input_events_path = "../data/external/inputevents.csv"
insulin_inputs_output = "../data/interim/insulin_inputs_in_ck_pacients.csv"
insulin_idxs = [223257, 223258, 223259, 223260, 223261, 223262, 229299, 229619]

In [8]:
chunksize = 10 ** 6

df_ck = pd.read_csv(ck_events_path)
all_pacients = df_ck['subject_id']

df_final = None
with pd.read_csv(input_events_path, chunksize=chunksize) as reader:
    for chunk in reader:
        df_pacients_with_ck = chunk[chunk['subject_id'].isin(all_pacients)]
        df_insulin_inputs = df_pacients_with_ck[df_pacients_with_ck['itemid'].isin(insulin_idxs)]
        if len(chunk) > 0:
            if df_final is None:
                df_final = df_insulin_inputs
            else:
                df_final = pd.concat([df_final,df_insulin_inputs])


print("Saving processed table in:", insulin_inputs_output)
df_final.to_csv(insulin_inputs_output)

Saving processed table in: ../data/interim/insulin_inputs_in_ck_pacients.csv


In [9]:
#visualizando a tabela gerada
df = pd.read_csv(insulin_inputs_output)
print(df.head())
print("LEN:", len(df))

   Unnamed: 0  Unnamed: 0.1  subject_id   hadm_id   stay_id  caregiver_id  \
0           0           772    10002428  23473524  35479615         41151   
1           1           780    10002428  23473524  35479615         41151   
2           2           859    10002428  23473524  35479615         74793   
3           3           993    10002428  23473524  35479615         98628   
4           4          2842    10004235  24181354  34100191         24745   

             starttime              endtime            storetime  itemid  ...  \
0  2156-05-19 22:41:00  2156-05-19 22:42:00  2156-05-19 22:42:00  223262  ...   
1  2156-05-20 04:00:00  2156-05-20 04:01:00  2156-05-20 06:05:00  223262  ...   
2  2156-05-13 09:43:00  2156-05-13 09:44:00  2156-05-13 09:43:00  223258  ...   
3  2156-05-13 16:00:00  2156-05-13 16:01:00  2156-05-13 17:16:00  223258  ...   
4  2196-02-25 00:54:00  2196-02-25 04:12:00  2196-02-25 00:54:00  223258  ...   

   ordercomponenttypedescription ordercategorydesc